# Modeling notebook

In [5]:
import pandas as pd
import numpy as np

testing_models = True

forecast_horizon = 547

from beepy import beep

# set benchmark with fourier series.

In [6]:
%run setup.ipynb

# tune models on a subset of categories

In [7]:

# subset for testing

if testing_models:
    train = train[train.store_item.isin(['1-1', '1-2', '1-3', '3-5', '6-4'])]
    test = test[test.store_item.isin(['1-1', '1-2', '1-3', '3-5', '6-4'])]
else: 
    from warnings import filterwarnings
    filterwarnings('ignore')
    

# Create df to populate with predictions

In [8]:
from datetime import timedelta

date_list = [train.index[-1] + timedelta(days=x+1) for x in range(forecast_horizon)]
horizon_end_date = date_list[-1]
fcast_begin = date_list[0]

In [9]:
store_item, dates = [], []
for series in train.store_item.unique():
    store_item.append(np.repeat(series, len(date_list)))
    dates.append(date_list)

In [10]:
predictions = pd.DataFrame(
    {
        'store_item': [x for sub in store_item for x in sub],
        'sales': test.sales
    }, index=[x for sub in dates for x in sub]
    )

# Fourier series (benchmark)

In [11]:

# https://notebook.community/statsmodels/statsmodels.github.io/devel/examples/notebooks/generated/deterministics
# from statsmodels.tsa.deterministic import Fourier, Seasonality, TimeTrend
# from statsmodels.tsa.deterministic import DeterministicProcess

# index = temp.index
# tt = TimeTrend(constant=True)
# four = Fourier(period=365.25, order=2)
# seas = Seasonality(period=7)
# det_proc = DeterministicProcess(index, additional_terms=[tt, seas, four])
# det_proc.in_sample().head(28)

In [12]:

from statsmodels.tsa.deterministic import Fourier

fourier_gen = Fourier(11, order=2)

temp = test[test.store_item == "1-1"]
fourier_gen.in_sample(temp.index).sum(axis=1)
# fourier_gen.out_of_sample(365, index=temp.index)


date
2016-07-03    2.000000
2016-07-04    2.706941
2016-07-05    1.425936
2016-07-06   -0.393719
2016-07-07   -1.031247
                ...   
2017-12-27   -0.393719
2017-12-28   -1.031247
2017-12-29   -0.377148
2017-12-30    0.140669
2017-12-31   -0.563104
Length: 547, dtype: float64

# vector autoreg

In [13]:
from statsmodels.tsa.ar_model import AutoReg
from tqdm import tqdm # progressbar

In [14]:
def fit_autoreg(df, fcast, horizion_end):
    preds_autoreg, trouble_series = [], []
    for series in tqdm(df.store_item.unique()):

        temp = df[df.store_item == series]
        temp.index.freq = "d"
        
        try:
            yhat = AutoReg(temp.sales, lags=5, old_names=False, seasonal=True, period=365)\
                .fit()\
                .predict(start = fcast, end = horizion_end)
            preds_autoreg.append(yhat)
        except np.linalg.LinAlgError:
            trouble_series.append(series)
            print(f'series {series} error')
            
    return [x for sub in preds_autoreg for x in sub]  

In [15]:
predictions["autoreg"] = fit_autoreg(df=train, fcast=fcast_begin, horizion_end=horizon_end_date)

100%|██████████| 5/5 [00:00<00:00,  9.21it/s]


# exp smoothing

In [16]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [17]:
def exp_smooth_predictor(df, seas, fcast=fcast_begin, horizion_end=horizon_end_date):
    exp_smooth_preds = []
    trouble_series = []
    for series in tqdm(df.store_item.unique()):
        temp = df[df.store_item == series]
        temp.index.freq = "d"
        try:
            preds = ExponentialSmoothing(temp.sales,     
                seasonal_periods=365,
                trend="add",
                seasonal=seas,
                use_boxcox=True,
                initialization_method="estimated")\
            .fit()\
            .predict(start = fcast, end = horizion_end)
            exp_smooth_preds.append(preds)
        except ValueError:
            trouble_series.append(series)
            print(series)

    return [x for sub in exp_smooth_preds for x in sub]  

In [ ]:
predictions["exp_smooth"] = exp_smooth_predictor(df=train, seas="add")
# predictions["exp_smooth_multi"] = exp_smooth_predictor(df=train, seas="multiplicative") 
# multiplicative is slow and inaccurate

# autoregressive distributed lag ARDL

In [ ]:
from statsmodels.tsa.api import ARDL

In [ ]:
def ardl_predictor(df, fcast=fcast_begin, horizion_end=horizon_end_date):
    ardl_preds = []
    for series in df.store_item.unique():
        temp = df[df.store_item == series]
        temp.index.freq = "d"
        
        ardl_pred = ARDL(temp.sales, 365, period=365, trend="t")\
            .fit()\
            .predict(start = fcast, end = horizion_end)
        ardl_preds.append(ardl_pred)
    return [x for sub in ardl_preds for x in sub]  

In [ ]:
predictions["ardl"] = ardl_predictor(train)

# xgboost


In [ ]:
from xgboost import XGBRegressor
# pip install xgboost==0.80
# the latest version kept crashing on me

In [ ]:
from helper import create_features

In [ ]:
def xgb_predictor(df, date_list=date_list):
    X_pred = create_features(pd.DataFrame(date_list, columns=["date"]))

    reg = XGBRegressor(n_estimators=1000)
    xgb_preds = []
    for series in tqdm(df.store_item.unique()):
        temp = df[df.store_item == series]
        X = create_features(pd.DataFrame(temp.index, columns=["date"]))
        preds = reg.fit(X, temp.sales)\
            .predict(X_pred)
        xgb_preds.append(preds)
    return [x for sub in xgb_preds for x in sub] 

In [ ]:
predictions["xgb_preds"] = xgb_predictor(train)

100%|██████████| 3/3 [00:01<00:00,  2.74it/s]


# prophet model

In [ ]:
from prophet import Prophet
# https://www.youtube.com/watch?v=pOYAXv15r3A

Importing plotly failed. Interactive plots will not work.


In [ ]:
def strict_inputs(df):
    df = df.drop("store_item", axis=1)\
        .reset_index()
    df["unique_id"] = series
    df = df.rename(columns={"sales":"y", "date":"ds"})
    return df[["ds", "y", "unique_id"]]

In [ ]:
def prophet_predictor(df, date_list=date_list):
    prophet_preds = []
    for series in tqdm(df.store_item.unique()):  
        temp = df[df.store_item == series]
        temp = strict_inputs(temp)
        model = Prophet(daily_seasonality=True)
        model.fit(temp)
        pred_frame = model.make_future_dataframe(periods=len(date_list), include_history=False)
        preds = model.predict(pred_frame)
        prophet_preds.append(preds.yhat)
    return [x for sub in prophet_preds for x in sub] 

In [ ]:
predictions["prophet"] = prophet_predictor(df=train)


  0%|          | 0/3 [00:00<?, ?it/s]

Initial log joint probability = -23.8441
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      94       2304.99    0.00059656       212.127    4.31e-06       0.001      145  LS failed, Hessian reset 
      99       2305.17   0.000825136       89.4793      0.9746      0.9746      150   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     193        2305.9    0.00037808       161.031   2.206e-06       0.001      316  LS failed, Hessian reset 
     199       2306.19    0.00069264       73.7016      0.3052           1      323   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     243       2306.29   4.25779e-05       86.2633   5.088e-07       0.001      415  LS failed, Hessian reset 
     271       2306.29   1.51602e-07       79.5501           1           1      452   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is

 33%|███▎      | 1/3 [00:02<00:05,  2.86s/it]

Initial log joint probability = -34.5062
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2717.63   0.000201329       78.4211      0.7713      0.7713      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2721.23   0.000657679       64.9873      0.4638           1      262   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     266       2721.27   2.03817e-05       61.8377   4.043e-07       0.001      393  LS failed, Hessian reset 
     299       2721.27   6.35448e-06       61.8897       1.593      0.4364      432   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     365       2721.28   1.48875e-07       76.8861      0.3067           1      514   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


 67%|██████▋   | 2/3 [00:04<00:02,  2.25s/it]

Initial log joint probability = -20.7053
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      98       2482.26   0.000209279       112.769   1.462e-06       0.001      192  LS failed, Hessian reset 
      99       2482.31    0.00124962       83.2433          10           1      194   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     114        2482.4   9.02554e-05       51.7204   1.639e-06       0.001      247  LS failed, Hessian reset 
     136       2482.41   2.62744e-05        85.761   3.638e-07       0.001      312  LS failed, Hessian reset 
     159       2482.41   1.24874e-07       61.4959      0.8433      0.8433      349   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


100%|██████████| 3/3 [00:06<00:00,  2.12s/it]


# Neural prophet

In [ ]:
from neuralprophet import NeuralProphet
# https://neuralprophet.com/html/model/README.html

In [ ]:
m = NeuralProphet(n_forecasts=1)
def neural_prophet(df, date_list=date_list):
    m_preds = []
    for series in tqdm(df.store_item.unique()):
        temp = df[df.store_item == series]
        temp = strict_inputs(temp)
        temp = temp[["y", "ds"]]
        temp.index.freq = "d"
        m.fit(temp, freq="D")
        future = m.make_future_dataframe(temp, periods=len(date_list))
        forecast = m.predict(future)
        m_preds.append(forecast.yhat1)
    return [x for sub in m_preds for x in sub] 

In [ ]:
from neuralprophet import set_random_seed 
set_random_seed(0)

In [ ]:
predictions["neural_prophet"] = neural_prophet(train)

  0%|          | 0/3 [00:00<?, ?it/s]INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:NP.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO:NP.config:Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 170
INFO:NP.config:Auto-set epochs to 170
 94%|█████████▎| 239/255 [00:01<00:00, 143.55it/s]
INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.11E-02, min: 1.13E+00
INFO:NP.utils_torch:lr-range-test results: steep: 3.11E-02, min: 1.13E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.40E-01
INFO:NP.utils_torch:learning rate range test selected lr: 3.40E-01
100%|██████████| 3/3 [00:34<00:00, 11.60s/it]


In [ ]:
store_sales = pd.concat([train, predictions])

In [ ]:
if not testing_models:
    from datetime import date
    today = date.today()
    predictions.to_csv(f"../data/predictions/predictions-{today.month}-{today.day}.csv")

# plot results

In [ ]:
from beepy import beep
beep()